## Gradienter, backpropagation

Gradienterna i djupa nätverk är inte stabila vid numerisk derivering (beräkning av gradienterna). Två huvudsakliga beteenden observeras:
* Försvinnande gradient (Vanishing Gradient) -- gradienten går mot noll desto djupare i närverket algoritmen går. Alltså slutar nätverket lära sig och de djupa lagren slutar uppdateras. Nätverket konvergerar inte mot någon lösning.
* Exploderande gradient (Exploding Gradient) -- gradienten divergerar och därmed även den förutsagda lösningen. Dessa gradienter kan till och med vara periodiska eller kaotiska. Förekommer i allmänhet bara i _rekurrenta_ nätverk (RNN), även om undantag finns!

### Vad som händer matematiskt

Detta var, som så mycket annat, en empirisk upptäckt. Ingen matematik fanns som förklarade beteendet
och ledde till att DNN (djupa neurala nätverk) övergavs kring 2000. Xavier Glorot och Yoshua Bengio 
visade kring 2010 att kombinationen av den aktiveringsfunktion som var mest populär (sigmoid) och 
hur vikterna i nätverket initialiserades ledde till att variansen ökade i varje lager. Det var alltså
en kombination av en statistisk effekt och ett numerisk närmevärde.

<img src="../Data/sigmoid_sat.jpg" />



I allmänhet kan inte variansen för både gradienterna och grundsanningen hållas nere om inte alla lager har samma antal kopplingar in och ut, vilket är en ytterligare anledning varför vissa nätverk föredrar att hålla samma antal noder i alla lager. Men Glorot och Bengio hittade en bra kompromiss, som visar sig fungera bra nog i praktiken: så-kallad _Glorot initialisering_.

Men den tekniken gäller bara för sigmoida funktioner. För ReLU föredras _He-initialisering_ (även _Kaiming-initialisering_ efter forskaren Kaiming He). Dessa skiljer sig mest i skalfaktorer och vilken statistika som används för variansen (in, ut eller medel). En till vanlig initalisering är _LeCun_ initialisering, som oftast används med SELU men även just nätverk där antalet noder i alla lager är detsamma.

<img src="../Data/active_init.jpg">

In [1]:
import torch
import torch.nn as nn
# av historiska skäl fungerar initialisering lite konstigt i Torch
# bäst är att göra det explicit:

layer = nn.Linear(40,10)

## He-initialisation
nn.init.kaiming_uniform_(layer.weight)
nn.init.zeros_(layer.bias) # bias kan lika gärna börja på noll

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

### Aktiveringsfunktioner

Så vilken aktiveringsfunktion skall man välja?

I praktiken är ReLU väldigt snabb att beräkna och mättas inte för positiva värden (har nollskild gradient), men den har ett problem som kallas "döende ReLU" -- under träningen kan dessa aktiveringsfunktioner hamna i ett läga då de alltid får negativ input och alltså alltid ger tillbaka 0. Därefter är den noden permanent 0, och alltså "död". 

En lösning är LeakyReLU:

<img src="../Data/LeakReLU.jpg">

Även en variant där vinkeln är en parameter, PReLU, finns.

In [2]:
alpha = 0.2
model = nn.Sequential(
    nn.Linear(50, 40),
    nn.LeakyReLU(negative_slope=alpha)    
)
nn.init.kaiming_uniform_(model[0].weight, alpha, nonlinearity="leaky_relu")

Parameter containing:
tensor([[ 0.3284,  0.2172,  0.2044,  ..., -0.1514,  0.3106, -0.2085],
        [ 0.2626, -0.1736, -0.1652,  ...,  0.0633,  0.2117, -0.2278],
        [ 0.2743,  0.2329,  0.0864,  ..., -0.2240,  0.2207, -0.1044],
        ...,
        [-0.1555, -0.2045, -0.0535,  ..., -0.0955,  0.2273, -0.3180],
        [ 0.1182,  0.1127,  0.0417,  ...,  0.2725, -0.0416, -0.0705],
        [ 0.3118, -0.0245,  0.3074,  ..., -0.0928, -0.1516,  0.3248]],
       requires_grad=True)

#### GELU, Swish, SwiGLU, Mish, RELU²

<img src="../Data/GELU.jpg">

GELU, Mish och Swish finns som <code>nn.GELU</code>, <code>nn.Mish</code> och <code>nn.SiLU</code>

Se boken kapitel 11 för detaljer och hur man implementerar de andra.

Swish och ReLU i praktiken vanligast, men tanh är viktig för LLMer.

## Normalisering

### Batch normalization

Initialisering hjälper bara i början av träningen och inte hindrar inte döda noder från att dyka upp senare i träningen. En teknik för att åtgärda detta är normalisering. Eftersom vi har att göra med neurala nätverk så preprocessar vi inte datan med tex StandardScaler utan lägger helt enkelt till ett lager i nätverket som utför normaliseringen! Ett typiskt sådant lager är ett _Batch Normalization Layer_ -- ett BN-lager. Notera att detta bara opererar på varje _batch_, dvs stickprov, och inte på hela träningsdatan. Vi måste därför vara noggranna med att blanda 
datan så att vi undviker konstiga medelvärden i stickproven. Dessutom behöver vi hålla reda på ett glidande medelvärde för att ha någon chans att fånga populationsmedlet under evaluering! Om vi inte gör det kommer skaleringen bli helt tokig när vi använder nätverket. Pytorch håller reda på detta åt oss, men nu börjar det bli viktigt att använda `model.train()` och `model.eval()`!

In [3]:
mnist_model = nn.Sequential(
    nn.Flatten(),
    nn.BatchNorm1d(1*28*28), # bilderna i mnist fashion är 28x28, vi plattade till ovanför
    nn.Linear(1 * 28 * 28, 300),
    nn.ReLU(),
    nn.BatchNorm1d(300),
    nn.Linear(300, 100),
    nn.ReLU(),
    nn.BatchNorm1d(100),
    nn.Linear(100, 10)
)
for layer in mnist_model:
    if isinstance(layer, nn.Linear):
        nn.init.kaiming_uniform_(layer.weight)
        nn.init.zeros_(layer.bias)

# Detta kommer inte ha så stor effekt -- det är relevant i mycket djupare nätverk.

### Layer normalization

Istället för att normalisera över stickprovets dimension (dvs värdena) så normaliserar lagernormalisering över feature-dimensionen. Det kan tyckas förvånande att det fungerar, men 
då skall vi minnas att vi skalerar och normaliserar just för att features skall vara i liknande
skala. Till skilland från BN behöver LN inte hålla reda på glidande medel, utan lär sig distributionen under träningen. 

In [5]:
inputs = torch.randn(32, 3, 100, 200) # ett stickprov om 32 st 3-färgers 100x200 slumpade bilder
layer_norm = nn.LayerNorm([3, 100, 200])
result = layer_norm(inputs) # normaliserar över alla tre färgkanaler samtidigt
result.shape

torch.Size([32, 3, 100, 200])

### Dropout

En kraftfull teknik, som dock har ett visst pris när det gäller statistik under träningen, är att sätta en sannolikhet för att en nod skall vara med eller inte under träningen. Detta betyder i praktiken att varje träningspass sker på ett nytt nätverk, med andra kopplingar. Det innebär att val/train accuracy och loss inte längre är så betydelsefulla, utan vi måste använda separata tester på okänd data. 

Extra viktigt att komma ihåg att byta till `model.eval()` innan nätverket används -- annars blir det något slumpässigt del-nätverk som kör varje gång!

In [ ]:
drop_model = nn.Sequential(
    nn.Flatten(),
    nn.Dropout(p=0.2), 
    nn.Linear(1 * 28 * 28, 100), 
    nn.ReLU(),
    nn.Dropout(p=0.2), 
    nn.Linear(100, 100), 
    nn.ReLU(),
    nn.Dropout(p=0.2), 
    nn.Linear(100, 100), 
    nn.ReLU(),
    nn.Dropout(p=0.2), 
    nn.Linear(100, 10)
)

Se kapitel 11 för än annu bättre version, som använder Monte Carlo simulering för att bestämma dropout!

<img src="../Data/perf_guidelines.png">

Weight-decay är ungefär lika med $\mathcal{l}_2$ regularisering och är en hyper-parameter på `AdamW` optimeraren.

## Visualisering

[Cats vs Dogs](../Resources/keras/L4b.dogs_vs_cats.ipynb)

[Visualising cats vs dogs](../Resources/keras/L4c.visualizing_cnns.ipynb)

[Feature visualization with deep dream-like optimization](https://distill.pub/2017/feature-visualization/)

[Extension of the above, exploration of InceptionV1 (aka GoogLeNet)](https://distill.pub/2019/activation-atlas/)